In [2]:
# Import libraries and dependencies
import os
import pandas as pd
import numpy as np
import hvplot.pandas
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt

In [3]:
# Set the file path into csv_path and reading the csv file into lending_club_data
csv_path= os.path.join("Resources/lc_2016_2017.csv")
lending_club_data = pd.read_csv(csv_path, low_memory=False)

# Print the info just to verify the read_csv is reading the file properly 
lending_club_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 72 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           759338 non-null  int64  
 1   member_id                    0 non-null       float64
 2   loan_amnt                    759338 non-null  int64  
 3   funded_amnt                  759338 non-null  int64  
 4   funded_amnt_inv              759338 non-null  float64
 5   term                         759338 non-null  object 
 6   int_rate                     759338 non-null  float64
 7   installment                  759338 non-null  float64
 8   grade                        759338 non-null  object 
 9   sub_grade                    759338 non-null  object 
 10  emp_title                    708368 non-null  object 
 11  emp_length                   708975 non-null  object 
 12  home_ownership               759338 non-null  object 
 13 

In [4]:
# Cleaning uneccesary data (pt.1)

lending_club_data.drop(['id','member_id','desc','policy_code','issue_d','initial_list_status','application_type'],axis=1,inplace=True)

lending_club_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 65 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    759338 non-null  int64  
 1   funded_amnt                  759338 non-null  int64  
 2   funded_amnt_inv              759338 non-null  float64
 3   term                         759338 non-null  object 
 4   int_rate                     759338 non-null  float64
 5   installment                  759338 non-null  float64
 6   grade                        759338 non-null  object 
 7   sub_grade                    759338 non-null  object 
 8   emp_title                    708368 non-null  object 
 9   emp_length                   708975 non-null  object 
 10  home_ownership               759338 non-null  object 
 11  annual_inc                   759338 non-null  float64
 12  verification_status          759338 non-null  object 
 13 

In [5]:
# Cleaning uneccesary data (pt.2)

# Converting null values to 0, or None depending on the type of data

lending_club_data[['dti_joint',
                   'annual_inc_joint',
                   'il_util' , 
                   'mths_since_rcnt_il', 
                   'open_acc_6m', 
                   'open_il_12m', 
                   'open_il_24m', 
                   'total_bal_il', 
                   'inq_last_12m' , 
                   'open_rv_12m', 
                   'open_rv_24m', 
                   'max_bal_bc', 
                   'all_util', 
                   'inq_fi', 
                   'total_cu_tl' ]] = lending_club_data[['dti_joint',
                                                         'annual_inc_joint',
                                                         'il_util' , 
                                                         'mths_since_rcnt_il', 
                                                         'open_acc_6m', 
                                                         'open_il_12m', 
                                                         'open_il_24m', 
                                                         'total_bal_il', 
                                                         'inq_last_12m' , 
                                                         'open_rv_12m', 
                                                         'open_rv_24m', 
                                                         'max_bal_bc', 
                                                         'all_util', 
                                                         'inq_fi', 
                                                         'total_cu_tl']].fillna(0)


lending_club_data[['tot_coll_amt',
                   'tot_cur_bal',
                   'total_rev_hi_lim']] = lending_club_data[['total_rev_hi_lim', 
                                                             'tot_cur_bal', 
                                                             'tot_coll_amt']].fillna(0)

lending_club_data.verification_status_joint = lending_club_data.verification_status_joint.fillna('None')

# Converting NaN values to 0 or 1 for the following columns

lending_club_data.mths_since_last_record = lending_club_data.mths_since_last_record.apply(lambda x: 0 if pd.isna(x) else 1) 
lending_club_data.mths_since_last_major_derog = lending_club_data.mths_since_last_major_derog.apply(lambda x: 0 if pd.isna(x) else 1) 
lending_club_data.mths_since_last_delinq = lending_club_data.mths_since_last_delinq.apply(lambda x: 0 if pd.isna(x) else 1)

# Drop the following five columns

lending_club_data.drop(['next_pymnt_d' ,'last_pymnt_d','last_credit_pull_d'],axis=1,inplace=True) 

lending_club_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    759338 non-null  int64  
 1   funded_amnt                  759338 non-null  int64  
 2   funded_amnt_inv              759338 non-null  float64
 3   term                         759338 non-null  object 
 4   int_rate                     759338 non-null  float64
 5   installment                  759338 non-null  float64
 6   grade                        759338 non-null  object 
 7   sub_grade                    759338 non-null  object 
 8   emp_title                    708368 non-null  object 
 9   emp_length                   708975 non-null  object 
 10  home_ownership               759338 non-null  object 
 11  annual_inc                   759338 non-null  float64
 12  verification_status          759338 non-null  object 
 13 

In [6]:
# Save emp_length (Employment Length) unique values into emp_length_unique DataFrame

emp_length_unique = lending_club_data[['emp_length']].value_counts().to_frame('total')

emp_length_unique

,total
emp_length,
10+ years,257944
2 years,69494
< 1 year,62724
3 years,60843
1 year,50585
5 years,46139
4 years,45573
6 years,32616
8 years,28707


In [13]:
# Making a copy of the original DataFrame into a new one
demograph_lc_data = lending_club_data.copy()

# Replacing the XX in the last two digits from zip code with a 00
demograph_lc_data["zip_code"] = demograph_lc_data["zip_code"].str[:-2] + "00"

# Giving a more proper format to the emp_title and home_ownership columns
demograph_lc_data["emp_title"] = demograph_lc_data["emp_title"].str.title()
demograph_lc_data["home_ownership"] = demograph_lc_data["home_ownership"].str.title()

# Saving the relevant information for demographic analysis into a smaller DataFrame
demograph_lc_data = demograph_lc_data[["addr_state", 
                                       "emp_title", 
                                       "emp_length",
                                       "annual_inc",
                                       "home_ownership",
                                       "title"]].dropna()

# Renaming the columns for better presentation
demograph_lc_data = demograph_lc_data.rename(columns={"addr_state": "Address State",
                                                      "emp_title": "Employee Title",
                                                      "emp_length": "Employment Length",
                                                      "annual_inc": "Annual Income",
                                                      "home_ownership": "Home Ownership",
                                                      "title": "Title"
})

# Displaying data sample
demograph_lc_data.head()

,Address State,Employee Title,Employment Length,Annual Income,Home Ownership,Title
1,MA,Teacher,10+ years,94000.0,Mortgage,Debt consolidation
2,MA,Front Office,7 years,46350.0,Mortgage,Home improvement
3,CA,Manager,10+ years,44000.0,Rent,Car financing
4,MN,Paramedic,10+ years,85000.0,Mortgage,Debt consolidation
5,PA,Teacher,10+ years,70000.0,Mortgage,Debt consolidation


In [14]:
# Set the file path into csv_path and reading the csv file into us_zipcodes_dat
csv_path_zip= os.path.join("Resources/uszips.csv")
us_zipcodes_data = pd.read_csv(csv_path_zip, low_memory=False)

# Dropping unnecesary columns from the data
us_zipcodes_data = us_zipcodes_data[["state_id", 
                                     "lat", 
                                     "lng", 
                                     "city"]].dropna()

# Print the info just to verify the read_csv is reading the file properly 
us_zipcodes_data.head()

,state_id,lat,lng,city
0,PR,18.18027,-66.75266,Adjuntas
1,PR,18.36075,-67.17541,Aguada
2,PR,18.45744,-67.12225,Aguadilla
3,PR,18.16585,-66.93716,Maricao
4,PR,18.29110,-67.12243,Anasco


In [16]:
# Step 1: Merge the DataFrames based on state code or zip code
merged_data = pd.merge(demograph_lc_data, us_zipcodes_data, left_on='Address State', right_on='state_id', how='left')

# Step 2: (Optional) Aggregate data if necessary
# For example, calculating mean annual income per state
agg_data = merged_data.groupby('Address State')['Annual Income'].mean().reset_index()

# Step 3: Create the choropleth map using GeoPandas
# Load the shapefile for US states
us_states = gpd.read_file("path/to/us_states_shapefile.shp")

# Merge the shapefile with the aggregated data
merged_states = pd.merge(us_states, agg_data, left_on='state_code', right_on='Address State', how='left')

# Plot the choropleth map
fig, ax = plt.subplots(1, 1)
merged_states.plot(column='Annual Income', cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
plt.show()

MemoryError: Unable to allocate 5.72 GiB for an array with shape (767450002,) and data type int64